# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
csv_path = "cities.csv"
weather_df = pd.read_csv(csv_path)
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Russell,32.3502,-85.1999,74.91,90,40,8.05,US,1624133565
1,Chokurdakh,70.6333,147.9167,61.21,42,79,10.74,RU,1624133566
2,São João del Rei,-21.1356,-44.2617,63.21,77,100,10.25,BR,1624133566
3,Show Low,34.2542,-110.0298,97.07,13,1,6.91,US,1624133566
4,Saldanha,-33.0117,17.9442,65.44,44,16,8.23,ZA,1624133567


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)

In [10]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:

# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70)
crit_windSpeed = weather_df["Wind Speed"] < 15
crit_cloudiness = weather_df["Clouds"] == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,121,Talara,-4.5772,-81.2719,75.96,73,0,12.66,PE,1624133592
1,155,Makat,47.6469,53.3463,79.14,23,0,9.42,KZ,1624133601
2,178,Kozluk,38.1938,41.4885,77.25,18,0,2.04,TR,1624133606
3,212,Shahr-e Bābak,30.1165,55.1186,73.63,10,0,11.41,IR,1624133615
4,256,Rayevskaya,44.8319,37.5556,76.12,83,0,8.79,RU,1624133625
5,273,Bauska,56.4075,24.1906,73.15,86,0,7.74,LV,1624133629
6,285,Chaman,30.9236,66.4512,77.20,7,0,3.69,PK,1624133631
7,295,Ürümqi,43.8010,87.6005,74.35,33,0,6.71,CN,1624133634
8,308,Mitsamiouli,-11.3847,43.2844,77.40,72,0,12.28,KM,1624133636
9,432,Thessalon,46.2501,-83.5666,70.09,55,0,14.94,CA,1624133664


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:

hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
#hotel_df.head()

In [13]:
hotel_df = ideal_weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -4.1417773,
                    "lng": -81.1073598
                },
                "viewport": {
                    "northeast": {
                        "lat": -4.140515570107277,
                        "lng": -81.10581927010729
                    },
                    "southwest": {
                        "lat": -4.143215229892721,
                        "lng": -81.10851892989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Aranwa Vichayito Bungalows & Carpas",
            "opening_hours": {
                "open_now": false
            },
            "photos": [
                {
                    "height": 459,
                    "html_attributions": [
      

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 47.5375311,
                    "lng": 52.9784805
                },
                "viewport": {
                    "northeast": {
                        "lat": 47.53886462989272,
                        "lng": 52.97941002989273
                    },
                    "southwest": {
                        "lat": 47.53616497010728,
                        "lng": 52.97671037010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Gostinitsa Orpk \"Nauryz\"",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2322,
                    "html_attributions": [
                    

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 30.123526,
                    "lng": 55.11758
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.12489222989272,
                        "lng": 55.11886677989273
                    },
                    "southwest": {
                        "lat": 30.12219257010728,
                        "lng": 55.11616712010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "\u0647\u062a\u0644 \u0647\u0627\u0648\u0634\u06a9",
            "photos": [
                {
                    "height": 1920,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/1064259

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKud-xDRwY_NaAmrJ7uthaBy8Tz3Jf3N5UwlsCfKf22phxnoiQclnBv1MDFceEA8k4__BLPq3yPLhWM8JuabgHRKH4C3k0B0PauEBSYxVtdzb-qpbk0OhWZ_i99KUnx51PNH8cg1I-o4and6EomH1N4o_tgs2bnZhnmsnoM8r7hmMQspTpmmwJ6W5APDI0160SI7j1HDFYFPxkfLYEl0YfdG2Xtt_PRAYG5udzV__kej_kCj9sarxAS-sXHbDQ9JROJDMPqMkpEsse0w_6luBHjEWRGbYfbbkPXYvzOGhq7KFuRZ0jCpiGWNu3LSffjcm4QeZ9rhIHIU6PLHaZFvVuBNWEb8ZrOnDu0_0IiGpksPWy9OYOs1kJaJMc7SXyDpa4gdr1NTsZAXfvDhwXDHyOeq5SI8I6L3iiX3q9QVj120GLin-f-GUIDQ4kGqMmc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 44.8756496,
                    "lng": 37.3123276
                },
                "viewport": {
                    "northeast": {
                        "lat": 44.87699942989272,
                        "lng": 37.31367742989272
                    },
                    "southwest": {
                        "lat": 44.874299770

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLuCiUhJO1PB2tqqt6qIQFS752gFke8mQ4OVc9lK0965gAI8Ot62Ftm4YY_PKm8e0nJ_JdvOgrJTfPaLleiS2ERjWB__o3BPo8090IwouKUSgIZjfEZ-NsOZGb22huoZT2hE6v9dmME6KmyJQoFYa6cqH1z6Sr8nVAXigvp4k7h0ZoV3dtZjiXDgxCPmek0IR99H0IxPvGRJ5yP_lDF0v0U6qi9PiYKG5fxE-wuq2alXFC4qDMCvObDSgLZ7o-an4KHLUmMRT_0wDnY0VKPyO0n6s1PZkCpO0daWwEzU65Kn_8HosnMNDyE4Fqdfcd6jb_yZa7wc3qb-T484tld9N0XdxF8VjgV7sjOvag3RFc3SOQWFAAsfueg2e1hqAdq2Tm_rF-lv8ILsFAIFYcK8i1aFZUfsHEhJjB2HpqL1d4qWEFjh7SPoq749zWIGDs-",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 56.6929162,
                    "lng": 23.7864721
                },
                "viewport": {
                    "northeast": {
                        "lat": 56.69411547989272,
                        "lng": 23.78780592989272
                    },
                    "southwest": {
                        "lat": 56.691415820

...

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJZUtG-j3qdqFxJvN3crcGNPD4Xq7fXSHXQUXR1n8E1gL92AOZamLSKeUUhfxkg3K_DyGkYkIyUMvdl9UjxlLTwQoOxaNaPImJHx1AganNxPuiGKNIarflVSjDIOo9g-YZJVx_hftlkA4UD7wEWMOtj5s0ujcgpFJ2Tsq5kz_p6nqdMbZs-6Pb3cXwYKZfe1JjWHg9CtgYrBFxHKgor4OHDkRJxqT6lyjD2Y8syqigXV-Sf9afas6hnBq8n2BMy2sDFZTcAbh6QvjDVrvOoJFYwDx78LX5gDtcJ00DO-lXz2hG0dKEWiZVEq2CWYMDDpwT-Kg4QQGQtMouZPu32eCzZCZcjsDyYxHDx9KJ8vnqW3WcsAKM7QXVq0ZZFj0t7fmGNLBBLgscwBy_b0IpCQcj6ttVQEsgREkxDw-ty8MT5S27K85B3iWqyj45gFMHS",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.0614134,
                    "lng": 27.9869549
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.06253522989272,
                        "lng": 27.98840882989272
                    },
                    "southwest": {
                        "lat": 36.059835570

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKFBvGDsMnynceunCcqXzpW3XAwupeVGUpTg7kPUyrXThdKMe-Xn1v8qU6w5qCMrTl0Qi3Krcbe7qJW28V08t3v67Tu-FSM5Zz3YR7KUppOBcuh3D6hJzegdgxlYF8wyiTW4lVFoCw1v3q7c4IyvYw3kZ_55dW047u6_20P1Unxcz_GaJsxbdfx2HUcUS8rCAKiFb7zXUJ9k3ffcCbYlKHtFsUVpVnLX4MzoAe8QRmMHWPUBjL2fybFGOO0HFA8XaqnLr2_acXQROcKsTtWOfUZ_u_9SPIyExmRF1M4go4bS0aZmvR1zZUlfqqGrWg0Zy2PPl7HFJdgre9yaP2_22Bqz4AlJ5PPkwWjW4ipuVXmiKi1zPUL0L3Xp5BFdOdfZ5H5QO8XsynXeT3fp4vQ5O-V0hl6ayfLqPOnswLH0oN7I-kmPx5e-6heFp2YjeDg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 41.3321064,
                    "lng": 48.4989786
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.33351362989273,
                        "lng": 48.50003427989272
                    },
                    "southwest": {
                        "lat": 41.330813970

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJcbXh1XBl1D9lAwawBhH8tvSeEcl_dBZFpIK75xSmJbYGOJipEep9ndh_aBSpJlVXu-QL4gKElf2DKJr-XbPBrVsroaPLGtxdkO1ViplNdVTwjh-WJKTdJ4tsutUsuk3kfm3KMnznmgnpYLHPck1OLheVtUSspSXmv4VVEOEUmh10uyON5jRoIA7Vieg1ixcmc2ttZ5fXhD_r2C8EZcCCbjooSuH7NfqY_Z2yZyH1pAlAMbnAGEs4I_gVhD60Px7u175Xr8V25hrC1AGrlwV58VmGEThRCIQjKaCWSftSYiJ_vqW1h3-rKjbKH2k8dt-XDmSQ4weESNU5HGWOh2j3cgJ0kP7fNDDlajoGzAnf-6ISvLMhdl2qWKo507kz2A3DIrg3Zp1tanMujVFrDyovzCpkmNZ0dfQW9ufIVxFDSL1CRbaym-69LMVoNmvgd",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 60.16788259999999,
                    "lng": 24.9472221
                },
                "viewport": {
                    "northeast": {
                        "lat": 60.16923242989272,
                        "lng": 24.94857192989272
                    },
                    "southwest": {
                        "lat": 60.16

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIcFkxrMzaCsWf8ocxxs9DHhe9jsD9pz-jT6jqAX77E1FspHfUxdICOXH-X-WNJO4hYdQua45fnhYjIIbhxw9yK-iAo70Os4B7V02aDPBRd7MDAOV5ehEH_LJIOiIWLBM9--CoOG9UYPLkAIbmxECJcbTaDP66gUMCEvhH2oZoDH_oXuGNn6ENvMaMPpjTrlFIEStpMCz1dPoUFZnjUJw13Fzp51Ej5t0kL_hPn1zKOv7AVB4xX4CyQ0T31zuk5oXQUoIfzkcSw-LgjybxRcdFzIXw8ZR-yR4F2bpZEklBxt4zJy9GCTL6raADslDw-Bg_78zGOhdp_qYm9h5j7BKXv5Ih6B3KiuGug4aoMq7hdAwNIOJpTG_TrzPTmH-VcL1aMRr62813U8JuKW0G_RRTNvq6jPRuHxjuPOOu6OdddkJwpJ3kXJUERTKqQ8ITu",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 41.263727,
                    "lng": 80.246
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.26515667989272,
                        "lng": 80.24738747989272
                    },
                    "southwest": {
                        "lat": 41.26245702010728

,index,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
0,121,Talara,-4.5772,-81.2719,75.96,73,0,12.66,PE,1624133592,Aranwa Vichayito Bungalows & Carpas
1,155,Makat,47.6469,53.3463,79.14,23,0,9.42,KZ,1624133601,"Gostinitsa Orpk ""Nauryz"""
2,178,Kozluk,38.1938,41.4885,77.25,18,0,2.04,TR,1624133606,REAL KONAK HOTEL
3,212,Shahr-e Bābak,30.1165,55.1186,73.63,10,0,11.41,IR,1624133615,هتل هاوشک
4,256,Rayevskaya,44.8319,37.5556,76.12,83,0,8.79,RU,1624133625,Utosov
5,273,Bauska,56.4075,24.1906,73.15,86,0,7.74,LV,1624133629,Agate Hotel
6,285,Chaman,30.9236,66.4512,77.20,7,0,3.69,PK,1624133631,Al taqdeer hotel
7,295,Ürümqi,43.8010,87.6005,74.35,33,0,6.71,CN,1624133634,Hilton Urumqi
8,308,Mitsamiouli,-11.3847,43.2844,77.40,72,0,12.28,KM,1624133636,Golden Tulip Grande Comore Moroni Resort & Spa
9,432,Thessalon,46.2501,-83.5666,70.09,55,0,14.94,CA,1624133664,Carolyn Beach Inn & Restaurant


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
#<dt>Country</dt><dd>{Country}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig2 = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig2.add_layer(heat_layer)
fig2.add_layer(hotel_layer)
# Display figure
fig2

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…